#### For Selected data

####  Individual Connectivity


In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns

# Parameters
directory = 'D:\oslomet\Dev_data\Selected data'
sampling_freq = 3.90625  # Given sampling frequency
num_channels = 48  # For oxy, deoxy, and total hemoglobin respectively

# Function to segment data based on experimental design
def segment_data(df, fs=sampling_freq):
    segments = {
          'thumb_tapping': df.iloc[int(25*fs):int(45*fs)],
        'index_tapping': df.iloc[int(45*fs):int(65*fs)],
        'middle_tapping': df.iloc[int(65*fs):int(85*fs)],
        'ring_tapping': df.iloc[int(85*fs):int(105*fs)],
        'little_tapping': df.iloc[int(105*fs):int(125*fs)],
    }
    return segments


# Function to calculate correlation matrices for each segment
def calculate_correlation_matrices(segments):
    correlation_matrices = {}
    for segment_name, segment_data in segments.items():
        oxy_data = segment_data.iloc[:, :num_channels]
        oxy_data = oxy_data.loc[:, oxy_data.std() != 0]  # Removing constant columns
        correlation_matrix = np.corrcoef(oxy_data.T)  # Transpose for correct correlation calculation
        correlation_matrices[segment_name] = correlation_matrix
    return correlation_matrices

# Function to visualize correlation matrices with spaced channel numbers starting from 1
def visualize_correlation_matrices(correlation_matrices, subject_id):
    fig, axs = plt.subplots(1, len(correlation_matrices), figsize=(20, 4))
    for ax, (segment_name, correlation_matrix) in zip(axs, correlation_matrices.items()):
        sns.heatmap(correlation_matrix, ax=ax, cmap='coolwarm', vmin=-1, vmax=1)
        ax.set_title(f'{subject_id} - {segment_name}')
        
        # Adjust x-axis tick labels and locations
        tick_positions_x = np.arange(0.5, num_channels + 0.5, 3)
        tick_labels_x = np.arange(1, num_channels + 1, 3)
        ax.set_xticks(tick_positions_x)
        ax.set_xticklabels(tick_labels_x)
        
        # Adjust y-axis tick labels and locations
        tick_positions_y = np.arange(0.5, num_channels + 0.5, 3)
        tick_labels_y = np.arange(1, num_channels + 1, 3)
        ax.set_yticks(tick_positions_y)
        ax.set_yticklabels(tick_labels_y)
        
    plt.tight_layout()
    plt.show()


# Main analysis loop
subject_matrices = {}  # Store average matrices for group analysis
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        subject_id = filename.split('_')[0]  # Extract subject ID from filename
        df = pd.read_csv(file_path)
        segments = segment_data(df)
        correlation_matrices = calculate_correlation_matrices(segments)
        
        # Individual-level analysis: visualize for each subject
        visualize_correlation_matrices(correlation_matrices, subject_id)
        
        # Store matrices for group-level analysis
        subject_matrices[subject_id] = correlation_matrices


#### Mean connectivity


In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns

# Parameters
directory = 'D:\oslomet\Dev_data\Selected data'
sampling_freq = 3.90625  # Adjusted based on your description
num_channels = 48  # Number of channels for oxy, deoxy, and total hemoglobin

# Function to segment data based on experimental design
def segment_data(df, fs=sampling_freq):
    segments = {
        'thumb_tapping': df.iloc[int(25*fs):int(45*fs)],
        'index_tapping': df.iloc[int(45*fs):int(65*fs)],
        'middle_tapping': df.iloc[int(65*fs):int(85*fs)],
        'ring_tapping': df.iloc[int(85*fs):int(105*fs)],
        'little_tapping': df.iloc[int(105*fs):int(125*fs)],
    }
    return segments


# Function to calculate and filter correlation matrices for each segment
def calculate_correlation_matrices(segments, threshold=1e-10):
    correlation_matrices = {}
    for name, data in segments.items():
        filtered_data = data.loc[:, data.var(axis=0) > threshold]
        if not filtered_data.empty:
            # Ensure the number of channels is 48
            if filtered_data.shape[1] != num_channels:
                if filtered_data.shape[1] < num_channels:
                    # Pad with zeros
                    padding = np.zeros((filtered_data.shape[0], num_channels - filtered_data.shape[1]))
                    filtered_data = pd.concat([filtered_data, pd.DataFrame(padding)], axis=1)
                else:
                    # Truncate to 48 channels
                    filtered_data = filtered_data.iloc[:, :num_channels]
            corr_matrix = np.corrcoef(filtered_data.T)
            correlation_matrices[name] = corr_matrix
        else:
            correlation_matrices[name] = np.array([])  # Empty array for no data
    return correlation_matrices

# Process and average data for each subject
subject_averages = {}
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        subject_id = filename.split('R')[0]  # S01R01 becomes S01
        df = pd.read_csv(os.path.join(directory, filename))
        segments = segment_data(df)
        correlation_matrices = calculate_correlation_matrices(segments)

        if subject_id not in subject_averages:
            subject_averages[subject_id] = {key: [] for key in segments.keys()}
        for name in segments.keys():
            subject_averages[subject_id][name].append(correlation_matrices[name])

# Average matrices for each subject and segment, ensuring consistent shapes
for subject_id, segments in subject_averages.items():
    for name, matrices in segments.items():
        # Filter out empty matrices and ensure consistent shapes for averaging
        non_empty_matrices = [m for m in matrices if m.size > 0]
        if non_empty_matrices:
            # Ensure all matrices have the same shape
            matrix_shape = non_empty_matrices[0].shape
            consistent_matrices = [m for m in non_empty_matrices if m.shape == matrix_shape]
            if consistent_matrices:
                # Now we can safely calculate the average
                averaged_matrix = np.mean(consistent_matrices, axis=0)
                subject_averages[subject_id][name] = averaged_matrix
            else:
                subject_averages[subject_id][name] = None  # Mark as None if shapes are inconsistent
        else:
            subject_averages[subject_id][name] = None  # No valid data

def visualize_averaged_matrices(subject_id, segment_names):
    # Exclude initial rest and final rest segments
    segment_names = [seg_name for seg_name in segment_names if seg_name not in ['initial_rest', 'final_rest']]
    
    fig, axs = plt.subplots(1, len(segment_names), figsize=(20, 4))
    for ax, segment_name in zip(axs, segment_names):
        matrix = subject_averages[subject_id].get(segment_name)
        if matrix is not None:
            sns.heatmap(matrix, ax=ax, cmap='coolwarm', vmin=-1, vmax=1)
            ax.set_title(f'{subject_id} - {segment_name}')
            
            # Set tick positions and labels to show every 3rd channel number
            tick_positions = np.arange(2, num_channels, 3)  # Channels are 0-indexed, so position for channel 1 is 0, for channel 4 is 3, etc.
            tick_labels = np.arange(1, num_channels + 1, 3)  # Actual channel labels starting from 1 and incrementing by 3
            ax.set_xticks(tick_positions)
            ax.set_xticklabels(tick_labels)
            ax.set_yticks(tick_positions)
            ax.set_yticklabels(tick_labels)
        else:
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{subject_id} - {segment_name}')
            ax.set_xticks([])
            ax.set_yticks([])
            
    plt.tight_layout()
    plt.show()

# Example: Visualize for a specific subject across all segments
segments = ['initial_rest', 'thumb_tapping', 'index_tapping', 'middle_tapping', 'ring_tapping', 'little_tapping', 'final_rest']
for subject_id in subject_averages.keys():
    visualize_averaged_matrices(subject_id, segments)
   

#### Average integrated response


In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns

# Parameters
sampling_freq = 3.90625  # Sampling frequency in Hz
initial_rest = 25  # Initial rest in seconds before the tasks start
finger_tapping_duration = 20  # Duration for each finger tapping in seconds
n_fingers = 5  # Number of fingers
rest_between_fingers = 0
# Number of channels for each data type (oxy, deoxy, total)
num_channels = 48  # Total channels are 144, 48 for each type


# Preparing data storage for integrated responses
integrated_responses = {'HbO': np.zeros((48, n_fingers))}

directory = 'D:\oslomet\Dev_data\Selected data'

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct file path
        file_path = os.path.join(directory, filename)
        # Read the file
        df = pd.read_csv(file_path, header=None)
        # Assign column names
        df.columns = ['HbO_' + str(i+1) for i in range(48)] + ['HbR_' + str(i+1) for i in range(48)] + ['HbT_' + str(i+1) for i in range(48)]
        
        # Loop through each finger tapping task based on the experiment structure
        for finger_index in range(n_fingers):
            # Calculate start and end index for the current finger tapping task
            start_time = initial_rest + finger_index * (finger_tapping_duration + rest_between_fingers)
            start_idx = int(start_time * sampling_freq)
            end_idx = int((start_time + finger_tapping_duration) * sampling_freq)
            
            # Calculate and store integrated responses for HbO channels
            for i in range(48):
                col_name = 'HbO_' + str(i+1)
                integrated_responses['HbO'][i, finger_index] += np.trapz(df[col_name][start_idx:end_idx])

# Normalize the integrated responses by the number of files (assuming uniform contribution from each file)
integrated_responses['HbO'] /= len([name for name in os.listdir(directory) if name.endswith('.csv')])

# Heatmap of average integrated HbO responses
plt.figure(figsize=(12, 10))
sns.heatmap(integrated_responses['HbO'], annot=False, cmap='viridis', yticklabels=[f' {i+1}' for i in range(48)], 
            xticklabels=['Thumb', 'Index', 'Middle', 'Ring', 'Little'])
plt.title('Heatmap of Average Integrated HbO Responses for Finger Tapping Across All Channels')
plt.xlabel('Fingers')
plt.ylabel('Channel')
plt.show()


##### Flux changes for all paricipants in single plot


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Define your directory containing the CSV files
directory = 'D:\\oslomet\\Dev_data\\Selected data'  # Update with your directory path
sampling_freq = 3.90625  # Sampling frequency in Hz
initial_rest_duration = 25  # Initial rest in seconds before the tasks start
finger_tapping_duration = 20  # Duration for each finger tapping in seconds
n_fingers = 5  # Number of fingers
experiment_duration = 350  # Total experimental duration in seconds

# Number of channels for each data type (oxy, deoxy, total)
num_channels = 48  # Total channels are 144, 48 for each type

# Function to preprocess and calculate the difference signal for each finger tapping
def calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq):
    # Create an array to hold the mean signals for the oxy-deoxy difference
    difference_signals = np.zeros((num_channels, n_fingers))  # Channels x fingers array for the signal difference

    # Calculate start and end indices for finger tapping intervals
    start_idx = int(initial_rest_duration * sampling_freq)
    for finger in range(n_fingers):
        end_idx = start_idx + int(finger_tapping_duration * sampling_freq)

        # Correct slicing for oxy and deoxy columns
        mean_oxy = df.iloc[start_idx:end_idx, :num_channels].mean(axis=0)  # Oxy columns: first 48
        mean_deoxy = df.iloc[start_idx:end_idx, num_channels:num_channels*2].mean(axis=0)  # Deoxy columns: second set of 48

        # Calculate the difference: oxy - deoxy
        difference_signals[:, finger] = mean_oxy.values - mean_deoxy.values

        # Update start index for next finger
        start_idx = end_idx

    return difference_signals

# Function to plot the difference signals as a single graph for all finger tapping tasks
def plot_difference_signals(difference_signals, sampling_freq):
    fig, ax = plt.subplots(figsize=(12, 8))

    time_values = np.linspace(0, experiment_duration, num=num_channels)  # Time values from 0 to 350 seconds
    
    finger_names = ['Thumb', 'Index', 'Middle', 'Ring', 'Little']  # Actual names of the fingers
    
    for finger in range(n_fingers):
        ax.plot(time_values, difference_signals[:, finger], marker='o', label=finger_names[finger])

    ax.legend()
    ax.set_title('Signal Difference (Oxy - Deoxy) for All Fingers')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Signal Difference (Oxy - Deoxy)')

    plt.tight_layout()  # Adjust layout to prevent cutting off labels
    plt.show()

# Main function to execute the analysis
def main():
    all_difference_signals = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            difference_signals = calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq)
            all_difference_signals.append(difference_signals)
    
    # Average the difference signals across all files
    mean_difference_signals = np.mean(all_difference_signals, axis=0)

    # Plot all finger tapping tasks in a single graph
    plot_difference_signals(mean_difference_signals, sampling_freq)

if __name__ == "__main__":
    main()


#### Flux changes for all paricipants in different plots



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Define your directory containing the CSV files
directory = 'D:\\oslomet\\Dev_data\\Selected data'  # Update with your directory path
sampling_freq = 3.90625  # Sampling frequency in Hz
initial_rest_duration = 25  # Initial rest in seconds before the tasks start
finger_tapping_duration = 20  # Duration for each finger tapping in seconds
n_fingers = 5  # Number of fingers

# Number of channels for each data type (oxy, deoxy, total)
num_channels = 48  # Total channels are 144, 48 for each type

# Function to preprocess and calculate the difference signal for each finger tapping
def calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq):
    # Create an array to hold the mean signals for the oxy-deoxy difference
    difference_signals = np.zeros((num_channels, n_fingers))  # Channels x fingers array for the signal difference

    # Calculate start and end indices for finger tapping intervals
    start_idx = int(initial_rest_duration * sampling_freq)
    for finger in range(n_fingers):
        end_idx = start_idx + int(finger_tapping_duration * sampling_freq)

        # Correct slicing for oxy and deoxy columns
        mean_oxy = df.iloc[start_idx:end_idx, :num_channels].mean(axis=0)  # Oxy columns: first 48
        mean_deoxy = df.iloc[start_idx:end_idx, num_channels:num_channels*2].mean(axis=0)  # Deoxy columns: second set of 48

        # Calculate the difference: oxy - deoxy
        difference_signals[:, finger] = mean_oxy.values - mean_deoxy.values

        # Update start index for next finger
        start_idx = end_idx

    return difference_signals

# Function to plot the difference signals as line graphs for each finger tapping task
def plot_line_graphs_for_finger(difference_signals, finger):
    fig, ax = plt.subplots(figsize=(8, 6))

    channels = np.arange(1, num_channels+1)  # Channel numbers for x-axis
    ax.plot(channels, difference_signals[:, finger-1], marker='o')

    ax.set_title(f'Line Graph of Signal Difference (Oxy - Deoxy) for Finger {finger}')
    ax.set_xlabel('Channel Number')
    ax.set_ylabel('Signal Difference')

    plt.tight_layout()  # Adjust layout to prevent cutting off labels
    plt.show()

# Main function to execute the analysis
def main():
    all_difference_signals = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            difference_signals = calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq)
            all_difference_signals.append(difference_signals)
    
    # Average the difference signals across all files
    mean_difference_signals = np.mean(all_difference_signals, axis=0)

    # Plot the mean difference as line graphs for each finger tapping task
    for finger in range(1, n_fingers+1):
        plot_line_graphs_for_finger(mean_difference_signals, finger)

if __name__ == "__main__":
    main()

#### Average flux in terms of heatmap


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Define your directory containing the CSV files
directory = 'D:\\oslomet\\Dev_data\\Selected data'  # Update with your directory path
sampling_freq = 3.90625  # Sampling frequency in Hz
initial_rest_duration = 25  # Initial rest in seconds before the tasks start
finger_tapping_duration = 20  # Duration for each finger tapping in seconds
n_fingers = 5  # Number of fingers

# Number of channels for each data type (oxy, deoxy, total)
num_channels = 48  # Total channels are 144, 48 for each type

# Function to preprocess and calculate the difference signal for each finger tapping

def calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq):
    # Create an array to hold the mean signals for the oxy-deoxy difference
    difference_signals = np.zeros((num_channels, n_fingers))  # Channels x fingers array for the signal difference

    # Calculate start and end indices for finger tapping intervals
    start_idx = int(initial_rest_duration * sampling_freq)
    for finger in range(n_fingers):
        end_idx = start_idx + int(finger_tapping_duration * sampling_freq)

        # Correct slicing for oxy and deoxy columns
        mean_oxy = df.iloc[start_idx:end_idx, :num_channels].mean(axis=0)  # Oxy columns: first 48
        mean_deoxy = df.iloc[start_idx:end_idx, num_channels:num_channels*2].mean(axis=0)  # Deoxy columns: second set of 48

        # Calculate the difference: oxy - deoxy
        difference_signals[:, finger] = mean_oxy.values - mean_deoxy.values

        # Update start index for next finger
        start_idx = end_idx

    return difference_signals


# Function to plot the heatmap for the difference in signals
def plot_heatmap(difference_signals):
    fig, ax = plt.subplots(figsize=(12, 8))
    cax = ax.imshow(difference_signals, aspect='auto', cmap='bwr', interpolation='none')
    cbar = fig.colorbar(cax, ax=ax, orientation='vertical')
    cbar.set_label('Oxy - Deoxy Signal')

    # Set the finger labels for x-axis
    finger_labels = ['Thumb', 'Index', 'Middle', 'Ring', 'Little']
    ax.set_xticks(np.arange(len(finger_labels)))
    ax.set_xticklabels(finger_labels, rotation=45)  # Rotate labels if they overlap

    # Set the channel labels for y-axis
    ax.set_yticks(np.arange(difference_signals.shape[0]))
    ax.set_yticklabels([f' {i+1}' for i in range(difference_signals.shape[0])])

    ax.set_title('Heatmap of Signal Difference (Oxy - Deoxy) Across Channels and Fingers')
    ax.set_xlabel('Fingers')
    ax.set_ylabel('Channels')
    
    plt.tight_layout()  # Adjust layout to prevent cutting off labels
    plt.show()

# Main function to execute the analysis
def main():
    all_difference_signals = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            difference_signals = calculate_difference_signals(df, num_channels, n_fingers, initial_rest_duration, finger_tapping_duration, sampling_freq)
            all_difference_signals.append(difference_signals)
    
    # Average the difference signals across all files
    mean_difference_signals = np.mean(all_difference_signals, axis=0)

    # Plot the mean difference as a heatmap
    plot_heatmap(mean_difference_signals)

if __name__ == "__main__":
    main()


##### Time-frequency analysis


In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt  


# Directory containing the CSV files
directory = r'D:\oslomet\Dev_data\Selected data'

# Sampling frequency
sampling_freq = 3.90625  # Hz

# Frequency bands of interest
freq_bands = {
    'Cardiac': (0.6, 2),
    'Respiratory_Myogenic': (0.15, 0.6),
    'Neurogenic': (0.015, 0.04),
    'Endothelial': (0.003, 0.015)
}

# Define the wavelet
wavelet_name = 'cmor1.5-1.0'

# Initialize a dictionary to store data for each participant
participants_data = {}

# Loop over each file in the directory to organize data by participant
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Extract participant ID (assuming filename format like 'S02R01...')
        participant_id = filename[:3]  # Adjust this slice according to your filename format
        
        # Construct the full file path and read the CSV file
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, header=None)
        
        # Add data to the dictionary
        if participant_id not in participants_data:
            participants_data[participant_id] = []
        participants_data[participant_id].append(df.iloc[:, 0].values)  # Assuming data is in the first column

# Process and plot data for each participant
for participant_id, signals in participants_data.items():
    # Calculate the mean signal across trials for the participant
    mean_signal = np.mean(np.column_stack(signals), axis=1)
    
    # Perform the CWT on the averaged signal
    scales = np.arange(1, 500)  # Adjust as needed
    coefficients, frequencies = pywt.cwt(mean_signal, scales, wavelet_name, 1/sampling_freq)
    
    # Plot for each frequency band
    for band, (f_low, f_high) in freq_bands.items():
        # Find indices of frequencies within the desired band
        freq_indices = np.where((frequencies >= f_low) & (frequencies <= f_high))[0]
        band_coefficients = coefficients[freq_indices, :]
        
        plt.figure(figsize=(12, 4))
        plt.imshow(np.abs(band_coefficients), extent=[0, len(mean_signal) / sampling_freq, f_high, f_low], 
                   cmap='jet', aspect='auto', interpolation='nearest', origin='lower')
        plt.colorbar(label='Magnitude')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.title(f'{band} Band Time-Frequency Representation of Averaged Trials for {participant_id}')
        plt.tight_layout()
        plt.show()
        plt.close()


#### Pearson's correlation and t-test


####  hypothesis "There are some  common activation patterns of motor cortex for individual finger tapping while they have a unique neural network involved during these individual finger tapping"



In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns
from scipy.stats import pearsonr, ttest_ind
from matplotlib.colors import LinearSegmentedColormap

# Parameters
directory = 'D:\\oslomet\\Dev_data\\Selected data'
sampling_freq = 3.90625  # Sampling frequency
num_channels = 48  # Number of channels

# Function to segment data based on experimental design
def segment_data(df, fs=sampling_freq):
    segments = {
        'Thumb': df.iloc[int(25*fs):int(45*fs), :num_channels].mean().mean(),
        'Index': df.iloc[int(45*fs):int(65*fs), :num_channels].mean().mean(),
        'Middle': df.iloc[int(65*fs):int(85*fs), :num_channels].mean().mean(),
        'Ring': df.iloc[int(85*fs):int(105*fs), :num_channels].mean().mean(),
        'Little': df.iloc[int(105*fs):int(125*fs), :num_channels].mean().mean(),
    }
    return segments

# Initialize a DataFrame to store all mean activations
all_activations = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct file path
        file_path = os.path.join(directory, filename)
        # Read the file
        df = pd.read_csv(file_path, header=None)
        # Segment the data
        activation = segment_data(df)
        all_activations.append(activation)

# Convert list of dictionaries to DataFrame
activation_df = pd.DataFrame(all_activations)

# Compute and print correlation matrix
correlation_matrix = activation_df.corr()
print("Correlation Matrix:\n", correlation_matrix, "\n")

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Pearson Correlation Coefficients')
plt.show()

# Function to perform and print t-tests
def perform_t_tests(df):
    fingers = df.columns
    p_values = pd.DataFrame(index=fingers, columns=fingers)
    
    for i in fingers:
        for j in fingers:
            if i != j:
                _, p_value = ttest_ind(df[i], df[j])
                p_values.at[i, j] = p_value
            else:
                p_values.at[i, j] = np.nan
    print("T-Test P-Values:\n", p_values, "\n")
    return p_values

# Perform t-tests and print results
p_values = perform_t_tests(activation_df)

# Fill diagonal with a special value and create a custom colormap
np.fill_diagonal(p_values.values, -1)  # Fill diagonal with -1
viridis = sns.color_palette("viridis", as_cmap=True)
colors = ["white"] + list(viridis(np.linspace(0, 1, 256)))
custom_cmap = LinearSegmentedColormap.from_list("custom_viridis", colors, N=257)

plt.figure(figsize=(10, 8))
sns.heatmap(p_values.astype(float), annot=True, fmt=".3f", cmap=custom_cmap, cbar_kws={'label': 'P-Value'})
plt.title('P-Values of Correlations')
plt.show()


Correlation Matrix:
            Thumb     Index    Middle      Ring    Little
Thumb   1.000000 -0.235687 -0.211916 -0.001076  0.012651
Index  -0.235687  1.000000 -0.476806  0.351949 -0.366037
Middle -0.211916 -0.476806  1.000000 -0.717872  0.456362
Ring   -0.001076  0.351949 -0.717872  1.000000 -0.713910
Little  0.012651 -0.366037  0.456362 -0.713910  1.000000 

T-Test P-Values:
            Thumb     Index    Middle      Ring    Little
Thumb        NaN  0.149888  0.144235  0.675773   0.02267
Index   0.149888       NaN  0.969444  0.207653  0.367912
Middle  0.144235  0.969444       NaN  0.198229  0.326802
Ring    0.675773  0.207653  0.198229       NaN  0.023473
Little   0.02267  0.367912  0.326802  0.023473       NaN 



#### Connectogram plots


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

# Parameters
directory = r'D:\oslomet\Dev_data\Selected data'
sampling_freq = 3.90625
num_channels = 48
segment_names = ['thumb_tapping', 'index_tapping', 'middle_tapping', 'ring_tapping', 'little_tapping']

# Initialize the subject_averages dictionary to store correlation matrices for each segment
subject_averages = {name: [] for name in segment_names}


# Function to segment data based on experimental design
def segment_data(df, fs=sampling_freq):
    segments = {
        'thumb_tapping': df.iloc[int(25*fs):int(45*fs)],
        'index_tapping': df.iloc[int(45*fs):int(65*fs)],
        'middle_tapping': df.iloc[int(65*fs):int(85*fs)],
        'ring_tapping': df.iloc[int(85*fs):int(105*fs)],
        'little_tapping': df.iloc[int(105*fs):int(125*fs)],
    }
    return segments


# Function to calculate and filter correlation matrices for each segment
def calculate_correlation_matrices(segments, threshold=1e-10):
    correlation_matrices = {}
    for name, data in segments.items():
        filtered_data = data.loc[:, data.var(axis=0) > threshold]
        if not filtered_data.empty:
            # Ensure the number of channels is 48
            if filtered_data.shape[1] != num_channels:
                if filtered_data.shape[1] < num_channels:
                    # Pad with zeros
                    padding = np.zeros((filtered_data.shape[0], num_channels - filtered_data.shape[1]))
                    filtered_data = pd.concat([filtered_data, pd.DataFrame(padding)], axis=1)
                else:
                    # Truncate to 48 channels
                    filtered_data = filtered_data.iloc[:, :num_channels]
            corr_matrix = np.corrcoef(filtered_data.T)
            correlation_matrices[name] = corr_matrix
        else:
            correlation_matrices[name] = np.empty((num_channels, num_channels))
            correlation_matrices[name][:] = np.NaN
    return correlation_matrices



# Function to read and process a single file
def process_file(file_path):
    df = pd.read_csv(file_path, header=None)
    segments = segment_data(df)
    correlation_matrices = calculate_correlation_matrices(segments)
    return correlation_matrices

# Process all files and accumulate correlation matrices for each segment
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        correlation_matrices = process_file(file_path)
        for name, corr_matrix in correlation_matrices.items():
            if corr_matrix.size != 0:  # Only add non-empty matrices
                subject_averages[name].append(corr_matrix)

# Now compute the mean correlation matrix for each segment
mean_correlation_matrices = {}
for segment_name in segment_names:
    matrices = subject_averages[segment_name]
    if len(matrices) > 0:
        stacked_matrices = np.stack(matrices)
        mean_corr_matrix = np.nanmean(stacked_matrices, axis=0)
        mean_correlation_matrices[segment_name] = mean_corr_matrix
# Loop over each file in the directory and process
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        correlation_matrices = process_file(file_path)  # Process each file
        for name in segment_names:
            subject_averages[name].append(correlation_matrices[name])  # Store the results

# Calculate the mean correlation matrix for each segment across all subjects
mean_correlation_matrices = {}
for name in segment_names:
    # Stack all the correlation matrices for this segment
    matrices = np.stack(subject_averages[name])
    # Calculate the mean across all subjects (axis=0)
    mean_correlation_matrix = np.nanmean(matrices, axis=0)
    mean_correlation_matrices[name] = mean_correlation_matrix

# Plotting function
def plot_connectogram(correlation_matrix, labels, threshold=0.7, title='Connectivity Plot'):
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': 'polar'})
    num_nodes = len(labels)
    angles = np.linspace(0, 2 * np.pi, num_nodes, endpoint=False).tolist()
    
    # Define a radius for the markers and the circle connecting them
    marker_radius = 0.9
    label_radius = 1.05  # Radius for labels
    
    # Draw the connectivity lines
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if abs(correlation_matrix[i][j]) > threshold:
                ax.plot([angles[i], angles[j]], [marker_radius, marker_radius], '-', 
                        linewidth=2, alpha=0.5, color='blue')
    
    # Draw the outer circle connecting the markers
    ax.plot(angles, np.full_like(angles, marker_radius), 'k-', markersize=6, linewidth=0.7)

    # Draw the markers (bullets)
    for angle in angles:
        ax.plot(angle, marker_radius, 'o', markersize=6, markerfacecolor='black', 
                markeredgecolor='black', zorder=3)
    
    # Add the channel labels
    for label, angle in zip(labels, angles):
        ha = 'center'
        va = 'center' if angle == 0 or angle == np.pi else 'bottom' if angle < np.pi else 'top'
        rotation = angle if angle < np.pi else angle + np.pi
        ax.text(angle, label_radius, label, size=10, horizontalalignment=ha, 
                verticalalignment=va, rotation=np.degrees(rotation), rotation_mode='anchor')

    # Customize the plot appearance
    ax.set_rmax(1.1)
    ax.set_rticks([])  # Disable radial ticks
    ax.set_xticks([])  # Disable circular ticks
    ax.set_title(title, va='bottom', pad=40, fontweight='bold', fontsize=18)
    ax.spines['polar'].set_visible(False)  # Hide the polar spine

    plt.show()

# Now plot the average connectogram for each segment
for name, matrix in mean_correlation_matrices.items():
    plot_connectogram(matrix, labels=[f'Ch{i+1}' for i in range(num_channels)], threshold=0.7, title=f'Connectogram for {name}')


#### Pearson's analysis

##### Average hemodynamic response (GLM analysis)